In [ ]:
%reset

In [2]:
%run './Functions.ipynb'

/home/katya/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


# 0. Functions

In [33]:
#Transforming weights into a heatmap-like df

def make_tables(chunk, n):
    index = set([x[-1] for x in unique_mutations[chunk]])
    columns = set([int(x[:-1]) for x in unique_mutations[chunk]])
    df_0 = pd.DataFrame(index=sorted(index), columns=columns).astype(float) # dummy dfs for each of the entrance neurons
    df_1 = pd.DataFrame(index=sorted(index), columns=columns).astype(float)
    
    for i in range(len(train_weights[chunk][n])):
        position = unique_mutations[chunk][i][:-1]
        mutation = unique_mutations[chunk][i][-1]
        df_0[int(position)][mutation] = (train_weights[chunk][n][i][0]).astype(float)
        df_1[int(position)][mutation] = (train_weights[chunk][n][i][1]).astype(float)
        
    return df_0,df_1

In [34]:
def extract_differences(chunk, n):
    mask = {}
    df = {}
    
    mask[0],mask[1] = make_tables(chunk,n)
    for key in mask:
        mask[key][mask[key]>0]=1
        mask[key][mask[key]<0]=-1
    
    mask = (mask[0] - mask[1]).fillna(0)
    df[0],df[1] = make_tables(chunk,n)
    
    return (df[0][mask!=0]-df[1][mask!=0]).fillna(0)

In [35]:
def count_occurences(chunk,n):
    effects = {}
    for n in range(n):
        result = extract_differences(chunk, n)
        for i in result.index:
            for j in result.columns:
                if result[j].ix[i]!=0:
                    if str(j)+str(i) not in effects.keys():
                        effects[str(j)+str(i)] = []
                        effects[str(j)+str(i)].append(result[j].ix[i])
                    else:
                        effects[str(j)+str(i)].append(result[j].ix[i])
    return effects

# 1. Training models and extracting weights

### 1.1. Test conditions on one sample

In [28]:
chunk = 'S5'

data, labels = read_data(chunk)

train_weights[chunk]={}
cv = cross_validation.StratifiedKFold(labels, n_folds = 20, shuffle = True)
counter = 0

for train_idx, test_idx in cv:
    print (counter)
    X_train, y_train = data[train_idx,:], labels[train_idx]
    X_test, y_test = data[test_idx,:], labels[test_idx]

    model = Sequential()
    init = initializers.Orthogonal(gain=1.0, seed=None)

    model.add(Dense(2, input_dim=data.shape[1], activation='sigmoid', kernel_initializer=init))
    model.add(Dense(100, activation='sigmoid', kernel_initializer=init))
    model.add(Dense(1, activation='sigmoid', kernel_initializer=init))

    opt = optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=1e-08, decay=0.0)

    model.compile(optimizer=opt,
                  loss='mean_squared_error',
                  verbose=0)

    model.fit(X_train, y_train, epochs=500, batch_size=500, shuffle=True, verbose=0)
    train_weights[chunk][counter] = model.layers[0].get_weights()[0]
    counter += 1

/home/katya/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=20.
  % (min_labels, self.n_folds)), Warning)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [38]:
for chunk in ['S5']:
    print chunk
    for i in range(20):
        _,_,R,_,_ = stats.linregress(train_weights[chunk][i][:,0],train_weights[chunk][i][:,1])

        print chunk,i,R

S5
S5 0 0.240440462253
S5 1 0.250738901426
S5 2 0.2650773999
S5 3 0.277421680114
S5 4 -0.0740884483528
S5 5 0.276924692172
S5 6 0.391093456367
S5 7 -0.395858567387
S5 8 -0.134742180854
S5 9 0.211869312216
S5 10 0.207364638688
S5 11 0.3699454901
S5 12 0.258245815706
S5 13 -0.439798662858
S5 14 -0.130604731684
S5 15 0.0508628012192
S5 16 -0.499467209335
S5 17 0.480540452372
S5 18 -0.489251086085
S5 19 0.372092915496


In [37]:
toprint = {}
for chunk in ['S5']:
    print chunk
    d = count_occurences(chunk,20)
    for key in d:
        if len(d[key])>10:
            print key+'\t'+str(np.median([np.abs(x) for x in d[key]]))+'\t'+str(np.std([np.abs(x) for x in d[key]]))+'\t'+str(len(d[key]))
    print '\n'

S5
2T	0.543092928827	0.165919797132	11
25L	1.61543478072	0.448285557388	14
21A	2.54797014594	1.98931401822	14
25S	0.929264426231	0.253345615892	12
22S	2.54611752182	0.68615643076	11




### 1.2. Performing the operation on all the samples

In [ ]:
train_weights = {}

for chunk in chunks:
    print chunk
    data, labels = read_data(chunk)
    train_weights[chunk]={}
    cv = cross_validation.StratifiedKFold(labels, n_folds = 10, shuffle = True)
    counter = 0

    for train_idx, test_idx in cv:
        print (counter)
        X_train, y_train = data[train_idx,:], labels[train_idx]
        X_test, y_test = data[test_idx,:], labels[test_idx]

        model = Sequential()
        init = initializers.Orthogonal(gain=1.0, seed=None)

        model.add(Dense(2, input_dim=data.shape[1], activation='sigmoid', kernel_initializer=init))
        model.add(Dense(100, activation='sigmoid', kernel_initializer=init))
        model.add(Dense(1, activation='sigmoid', kernel_initializer=init))

        opt = optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=1e-08, decay=0.0)

        model.compile(optimizer=opt,
                      loss='mean_squared_error',
                      verbose=0)

        model.fit(X_train, y_train, epochs=500, batch_size=500, shuffle=True, verbose=0)
        train_weights[chunk][counter] = model.layers[0].get_weights()[0]
        counter += 1

S1
0


In [ ]:
toprint = {}
for chunk in chunks:
    print chunk
    d = count_occurences(chunk,10)
    for key in d:
        if len(d[key])>5:
            print key+'\t'+str(np.median([np.abs(x) for x in d[key]]))+'\t'+str(np.std([np.abs(x) for x in d[key]]))+'\t'+str(len(d[key]))
    print '\n'

In [ ]:
toprint = {}
for chunk in chunks:
    print chunk
    d = count_occurences(chunk,10)
    for key in d:
        if len(d[key])>5:
            print key+'\t'+str(np.median([np.abs(x) for x in d[key]]))+'\t'+str(np.std([np.abs(x) for x in d[key]]))+'\t'+str(len(d[key]))
    print '\n'

In [ ]:
for chunk in chunks:
    print chunk
    for i in range(10):
        _,_,R,_,_ = stats.linregress(train_weights[chunk][i][:,0],train_weights[chunk][i][:,1])
        if R < 0:
            print chunk,i,R